# Extract Custom Fields from Your File

This notebook demonstrates how to use analyzers to extract custom fields from your input files.

## Prerequisites
1. Ensure Azure AI service is configured following [steps](../README.md#configure-azure-ai-service-resource)
2. Install the required packages to run the sample.

In [1]:
%pip install -r ../requirements.txt

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Analyzer Templates

Below is a collection of analyzer templates designed to extract fields from various input file types.

These templates are highly customizable, allowing you to modify them to suit your specific needs. For additional verified templates from Microsoft, please visit [here](../analyzer_templates/README.md).

In [2]:
extraction_templates = {
    "invoice":            ('../analyzer_templates/invoice.json',         '../data/invoice.pdf'            ),
    "chart":              ('../analyzer_templates/image_chart.json',     '../data/pieChart.jpg'           ),
    "call_recording":     ('../analyzer_templates/call_recording_analytics.json', '../data/callCenterRecording.mp3'),
    "conversation_audio": ('../analyzer_templates/conversational_audio_analytics.json', '../data/callCenterRecording.mp3'),
    "marketing_video":    ('../analyzer_templates/marketing_video.json', '../data/video.mp4'              )
}

Specify the analyzer template you want to use and provide a name for the analyzer to be created based on the template.

In [3]:
import uuid

ANALYZER_TEMPLATE = "call_recording"
ANALYZER_ID = "field-extraction-sample-" + str(uuid.uuid4())

(analyzer_template_path, analyzer_sample_file_path) = extraction_templates[ANALYZER_TEMPLATE]

## Create Azure AI Content Understanding Client

> The [AzureContentUnderstandingClient](../python/content_understanding_client.py) is a utility class containing functions to interact with the Content Understanding API. Before the official release of the Content Understanding SDK, it can be regarded as a lightweight SDK.


In [4]:
import logging
import json
import os
import sys
from pathlib import Path
from dotenv import find_dotenv, load_dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

load_dotenv(find_dotenv())
logging.basicConfig(level=logging.INFO)

AZURE_AI_ENDPOINT = os.getenv("AZURE_AI_ENDPOINT")
AZURE_AI_API_VERSION = os.getenv("AZURE_AI_API_VERSION", "2024-12-01-preview")

# Add the parent directory to the path to use shared modules
parent_dir = Path(Path.cwd()).parent
sys.path.append(str(parent_dir))
from python.content_understanding_client import AzureContentUnderstandingClient

credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")

client = AzureContentUnderstandingClient(
    endpoint=AZURE_AI_ENDPOINT,
    api_version=AZURE_AI_API_VERSION,
    token_provider=token_provider,
    x_ms_useragent="azure-ai-content-understanding-python/field_extraction", # This header is used for sample usage telemetry, please comment out this line if you want to opt out.
)

INFO:azure.identity._credentials.environment:No environment configuration found.
INFO:azure.identity._credentials.managed_identity:ManagedIdentityCredential will use IMDS
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'http://169.254.169.254/metadata/identity/oauth2/token?api-version=REDACTED&resource=REDACTED'
Request method: 'GET'
Request headers:
    'User-Agent': 'azsdk-python-identity/1.19.0 Python/3.11.11 (Linux-6.5.0-1025-azure-x86_64-with-glibc2.36)'
No body was attached to the request
INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 400
Response headers:
    'Content-Type': 'application/json; charset=utf-8'
    'Server': 'IMDS/150.870.65.1475'
    'x-ms-request-id': '984d1c76-2999-468c-bf80-b9e0e9f5d2dd'
    'Date': 'Wed, 12 Feb 2025 23:08:30 GMT'
    'Content-Length': '88'
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'http://169.254.169.254/metadata/identity/oauth2/token?api-version=REDACTED&resource=REDACTED'
Re

## Create Analyzer from the Template

In [5]:
response = client.begin_create_analyzer(ANALYZER_ID, analyzer_template_path=analyzer_template_path)
result = client.poll_result(response)

print(json.dumps(result, indent=2))

INFO:python.content_understanding_client:Analyzer field-extraction-sample-3a0134db-6f67-433c-b0fc-d4e6e8c977ac create request accepted.
INFO:python.content_understanding_client:Request 6c6c740b-3c3e-460c-af22-d6819674f3e6 in progress ...
INFO:python.content_understanding_client:Request result is ready after 2.21 seconds.


{
  "id": "6c6c740b-3c3e-460c-af22-d6819674f3e6",
  "status": "Succeeded",
  "result": {
    "analyzerId": "field-extraction-sample-3a0134db-6f67-433c-b0fc-d4e6e8c977ac",
    "description": "Sample call recording analytics",
    "createdAt": "2025-02-12T23:08:32Z",
    "lastModifiedAt": "2025-02-12T23:08:33Z",
    "config": {
      "locales": [
        "en-US"
      ],
      "returnDetails": true,
      "disableContentFiltering": false
    },
    "fieldSchema": {
      "fields": {
        "Summary": {
          "type": "string",
          "method": "generate",
          "description": "A one-paragraph summary"
        },
        "Topics": {
          "type": "array",
          "method": "generate",
          "description": "Top 5 topics mentioned",
          "items": {
            "type": "string"
          }
        },
        "Companies": {
          "type": "array",
          "method": "generate",
          "description": "List of companies mentioned",
          "items": {
         

## Extract Fields Using the Analyzer

After the analyzer is successfully created, we can use it to analyze our input files.

In [6]:
response = client.begin_analyze(ANALYZER_ID, file_location=analyzer_sample_file_path)
result = client.poll_result(response)

print(json.dumps(result, indent=2))

INFO:python.content_understanding_client:Analyzing file ../data/callCenterRecording.mp3 with analyzer: field-extraction-sample-3a0134db-6f67-433c-b0fc-d4e6e8c977ac
INFO:python.content_understanding_client:Request ad53aa68-5128-416d-90b8-4d3e5d287179 in progress ...
INFO:python.content_understanding_client:Request ad53aa68-5128-416d-90b8-4d3e5d287179 in progress ...
INFO:python.content_understanding_client:Request result is ready after 4.32 seconds.


{
  "id": "ad53aa68-5128-416d-90b8-4d3e5d287179",
  "status": "Succeeded",
  "result": {
    "analyzerId": "field-extraction-sample-3a0134db-6f67-433c-b0fc-d4e6e8c977ac",
    "apiVersion": "2024-12-01-preview",
    "createdAt": "2025-02-12T23:08:34Z",
    "warnings": [],
    "contents": [
      {
        "markdown": "```WEBVTT\n\n00:00.080 --> 00:00.640\n<v Agent>Good day.\n\n00:00.960 --> 00:02.240\n<v Agent>Welcome to Contoso.\n\n00:02.560 --> 00:03.760\n<v Agent>My name is John Doe.\n\n00:03.920 --> 00:05.120\n<v Agent>How can I help you today?\n\n00:05.440 --> 00:06.320\n<v Customer>Yes, good day.\n\n00:06.720 --> 00:08.160\n<v Customer>My name is Maria Smith.\n\n00:08.560 --> 00:11.280\n<v Customer>I would like to inquire about my current point balance.\n\n00:11.680 --> 00:12.560\n<v Agent>No problem.\n\n00:12.880 --> 00:13.920\n<v Agent>I am happy to help.\n\n00:14.240 --> 00:16.720\n<v Agent>I need your date of birth to confirm your identity.\n\n00:17.120 --> 00:19.600\n<v Custo

## Clean Up
Optionally, delete the sample analyzer from your resource. In typical usage scenarios, you would analyze multiple files using the same analyzer.

In [7]:
client.delete_analyzer(ANALYZER_ID)

INFO:python.content_understanding_client:Analyzer field-extraction-sample-3a0134db-6f67-433c-b0fc-d4e6e8c977ac deleted.


<Response [204]>